# (윤성님이 보내주신) "홈쇼핑+시청률" 데이터 셋에 "기상" 데이터 결합

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm

In [2]:
homeshop_rating = pd.read_csv('C:/Users/Playdata/2020_Bigcontest/dataset/homeshop_rating.csv')
homeshop_rating

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2,0.0000
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2,0.0000
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2,0.0000
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2,0.0000
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2,0.0278
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2,0.0278
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2,0.0278


In [3]:
weather = pd.read_csv('C:/Users/Playdata/2020_Bigcontest/dataset/weather/weather_final.csv')
weather

,date,temp,rain,humidity,snow,dust,is_rain,is_snow
0,2019-01-01 00:00:00,-5.5,NaN,54.0,NaN,59.0,0,0
1,2019-01-01 01:00:00,-5.9,NaN,56.0,NaN,58.0,0,0
2,2019-01-01 02:00:00,-6.5,NaN,60.0,NaN,44.0,0,0
3,2019-01-01 03:00:00,-6.9,NaN,62.0,NaN,48.0,0,0
4,2019-01-01 04:00:00,-7.2,NaN,57.0,NaN,51.0,0,0
...,...,...,...,...,...,...,...,...
8756,2019-12-31 20:00:00,-7.0,NaN,41.0,NaN,27.0,0,0
8757,2019-12-31 21:00:00,-7.4,NaN,43.0,NaN,27.0,0,0
8758,2019-12-31 22:00:00,-8.2,NaN,42.0,NaN,29.0,0,0
8759,2019-12-31 23:00:00,-7.4,NaN,45.0,NaN,24.0,0,0


- 결합을 해주기 전에 각 데이터 셋의 변수 타입을 확인해봐야 한다.
    - 조인키로 사용할 date 변수의 변수 타입을 동일하게 해주어야 하기 때문!!

In [4]:
homeshop_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      37372 non-null  object 
 1   exp_mins  37372 non-null  float64
 2   mom_code  37372 non-null  int64  
 3   pd_code   37372 non-null  int64  
 4   pd_name   37372 non-null  object 
 5   pd_group  37372 non-null  object 
 6   pd_price  37372 non-null  int64  
 7   sales     37372 non-null  float64
 8   weekdays  37372 non-null  object 
 9   seasons   37372 non-null  int64  
 10  rating    37372 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 3.1+ MB


In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      8761 non-null   object 
 1   temp      8761 non-null   float64
 2   rain      824 non-null    float64
 3   humidity  8761 non-null   float64
 4   snow      82 non-null     float64
 5   dust      8200 non-null   float64
 6   is_rain   8761 non-null   int64  
 7   is_snow   8761 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 547.7+ KB


- 각 데이터 셋에서 date 변수를 활용해서 **"년-월-일-시간" 형태의 새로운 변수(new_date)**를 만들어주겠다.
    - 그 다음, new_date 변수를 조인키로 결합을 시켜줄 것이다.
- 그러기 위해서 먼저, 각 데이터 셋의 date 변수의 변수 타입을 datetime으로 변경해주도록 하겠다.

In [6]:
homeshop_rating['date'] = pd.to_datetime(homeshop_rating['date'], format =  '%Y-%m-%d %H:%M:%S')
weather['date'] = pd.to_datetime(weather['date'], format =  '%Y-%m-%d %H:%M:%S')

In [7]:
homeshop_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      37372 non-null  datetime64[ns]
 1   exp_mins  37372 non-null  float64       
 2   mom_code  37372 non-null  int64         
 3   pd_code   37372 non-null  int64         
 4   pd_name   37372 non-null  object        
 5   pd_group  37372 non-null  object        
 6   pd_price  37372 non-null  int64         
 7   sales     37372 non-null  float64       
 8   weekdays  37372 non-null  object        
 9   seasons   37372 non-null  int64         
 10  rating    37372 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(3)
memory usage: 3.1+ MB


In [8]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      8761 non-null   datetime64[ns]
 1   temp      8761 non-null   float64       
 2   rain      824 non-null    float64       
 3   humidity  8761 non-null   float64       
 4   snow      82 non-null     float64       
 5   dust      8200 non-null   float64       
 6   is_rain   8761 non-null   int64         
 7   is_snow   8761 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 547.7 KB


In [9]:
# 년(year), 월(month), 일(day), 시간(hour) 변수 생성
homeshop_rating['year'] = homeshop_rating['date'].dt.year
homeshop_rating['month'] = homeshop_rating['date'].dt.month
homeshop_rating['day'] = homeshop_rating['date'].dt.day
homeshop_rating['hour'] = homeshop_rating['date'].dt.hour

weather['year'] = weather['date'].dt.year
weather['month'] = weather['date'].dt.month
weather['day'] = weather['date'].dt.day
weather['hour'] = weather['date'].dt.hour

In [10]:
# 아래의 for문을 돌리기 위해 임시적으로 "0" 값으로 채워진 new_date 변수를 미리 만들어 주었음
homeshop_rating['new_date'] = 0

# "년-월-일-시간" 형태의 새로운 변수 new_date 생성
for i in tqdm(range(len(homeshop_rating))):
    homeshop_rating['new_date'][i] = str(homeshop_rating['year'][i]) + '-' + str(homeshop_rating['month'][i]) + '-' + \
                                     str(homeshop_rating['day'][i]) + '-' + str(homeshop_rating['hour'][i])
homeshop_rating

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,year,month,day,hour,new_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2,0.0000,2019,1,1,6,2019-1-1-6
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2,0.0000,2019,1,1,6,2019-1-1-6
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2,0.0000,2019,1,1,6,2019-1-1-6
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2,0.0000,2019,1,1,6,2019-1-1-6
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2,0.0000,2019,1,1,6,2019-1-1-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218,2019,12,31,23,2019-12-31-23
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2,0.0278,2020,1,1,0,2020-1-1-0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2,0.0278,2020,1,1,0,2020-1-1-0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2,0.0278,2020,1,1,0,2020-1-1-0


In [11]:
# 아래의 for문을 돌리기 위해 임시적으로 "0" 값으로 채워진 new_date 변수를 미리 만들어 주었음
weather['new_date'] = 0

# "년-월-일-시간" 형태의 새로운 변수 new_date 생성
for i in tqdm(range(len(weather))):
    weather['new_date'][i] = str(weather['year'][i]) + '-' + str(weather['month'][i]) + '-' + \
                             str(weather['day'][i]) + '-' + str(weather['hour'][i])
weather

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,temp,rain,humidity,snow,dust,is_rain,is_snow,year,month,day,hour,new_date
0,2019-01-01 00:00:00,-5.5,NaN,54.0,NaN,59.0,0,0,2019,1,1,0,2019-1-1-0
1,2019-01-01 01:00:00,-5.9,NaN,56.0,NaN,58.0,0,0,2019,1,1,1,2019-1-1-1
2,2019-01-01 02:00:00,-6.5,NaN,60.0,NaN,44.0,0,0,2019,1,1,2,2019-1-1-2
3,2019-01-01 03:00:00,-6.9,NaN,62.0,NaN,48.0,0,0,2019,1,1,3,2019-1-1-3
4,2019-01-01 04:00:00,-7.2,NaN,57.0,NaN,51.0,0,0,2019,1,1,4,2019-1-1-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2019-12-31 20:00:00,-7.0,NaN,41.0,NaN,27.0,0,0,2019,12,31,20,2019-12-31-20
8757,2019-12-31 21:00:00,-7.4,NaN,43.0,NaN,27.0,0,0,2019,12,31,21,2019-12-31-21
8758,2019-12-31 22:00:00,-8.2,NaN,42.0,NaN,29.0,0,0,2019,12,31,22,2019-12-31-22
8759,2019-12-31 23:00:00,-7.4,NaN,45.0,NaN,24.0,0,0,2019,12,31,23,2019-12-31-23


- 결합을 위한 조인키(new_date)를 만들어주었으므로, 이제 불필요한 변수인 **"year, month, day, hour" 변수는 제거**하도록 하겠다.
    - 추가적으로 **date라는 변수가 두 개의 데이터 셋에 모두 존재**하므로, **weather 데이터 셋의 date 변수는 제거**해주겠다.

In [12]:
homeshop_rating.drop(['year', 'month', 'day', 'hour'], axis = 1, inplace = True)
weather.drop(['date', 'year', 'month', 'day', 'hour'], axis = 1, inplace = True)

In [13]:
homeshop_rating.head()

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,new_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2,0.0,2019-1-1-6
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2,0.0,2019-1-1-6
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2,0.0,2019-1-1-6
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2,0.0,2019-1-1-6
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2,0.0,2019-1-1-6


In [14]:
weather.head()

,temp,rain,humidity,snow,dust,is_rain,is_snow,new_date
0,-5.5,NaN,54.0,NaN,59.0,0,0,2019-1-1-0
1,-5.9,NaN,56.0,NaN,58.0,0,0,2019-1-1-1
2,-6.5,NaN,60.0,NaN,44.0,0,0,2019-1-1-2
3,-6.9,NaN,62.0,NaN,48.0,0,0,2019-1-1-3
4,-7.2,NaN,57.0,NaN,51.0,0,0,2019-1-1-4


- (중복된 값을 제외한) 두 데이터 셋의 new_date 개수가 서로 불일치 하기 때문에, **inner join**을 통해 "홈쇼핑+시청률" 데이터 셋에 존재하는 new_date에 대해서만 결합을 수행해주도록 하겠다.
    - 즉, **new_date 변수를 조인키로 inner join** 해주겠다!!

In [15]:
df_final = pd.merge(homeshop_rating, weather, how = 'inner', on = 'new_date')
df_final

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,new_date,temp,rain,humidity,snow,dust,is_rain,is_snow
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218,2019-12-31-23,-7.4,NaN,45.0,NaN,24.0,0,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2,0.0278,2020-1-1-0,-6.5,0.0,38.0,NaN,27.0,0,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2,0.0278,2020-1-1-0,-6.5,0.0,38.0,NaN,27.0,0,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2,0.0278,2020-1-1-0,-6.5,0.0,38.0,NaN,27.0,0,0


- Inner join이 성공적으로 잘 수행되었으니, 이제 불필요한 new_date 변수는 제거해주도록 하겠다.

In [16]:
df_final.drop(['new_date'], axis = 1, inplace = True)
df_final

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,temp,rain,humidity,snow,dust,is_rain,is_snow
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,2,0.0000,-7.9,NaN,60.0,NaN,65.0,0,0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,2,0.0000,-7.9,NaN,60.0,NaN,65.0,0,0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,2,0.0000,-7.9,NaN,60.0,NaN,65.0,0,0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,2,0.0000,-7.9,NaN,60.0,NaN,65.0,0,0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,2,0.0000,-7.9,NaN,60.0,NaN,65.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218,-7.4,NaN,45.0,NaN,24.0,0,0
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,2,0.0278,-6.5,0.0,38.0,NaN,27.0,0,0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,2,0.0278,-6.5,0.0,38.0,NaN,27.0,0,0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,2,0.0278,-6.5,0.0,38.0,NaN,27.0,0,0


# 최종 결합된 데이터 셋 csv 파일로 내보내기

In [17]:
df_final.to_csv('C:/Users/Playdata/2020_Bigcontest/dataset/df_final.csv', index = False)